In [ ]:
import random
import math


class GA:
    def __init__(self, individualSize, populationSize):
        self.population = dict()
        self.individualSize = individualSize
        self.populationSize = populationSize
        self.totalFitness = 0

        i = 0
        while i < populationSize:
            listOfBits = [0] * individualSize
            listOfLocations = list(range(0, individualSize))
            numberOfOnes = random.randint(0, individualSize - 1)
            onesLocations = random.sample(listOfLocations, numberOfOnes)
            for j in onesLocations:
                listOfBits[j] = 1
                self.population[i] = [listOfBits, numberOfOnes]
                self.totalFitness = self.totalFitness + numberOfOnes
                i += 1

    def updatePopulationFitness(self):
        self.totalFitness = 0
        for individual in self.population:
            individualFitness = sum(self.population[individual][0])
            self.population[individual][1] = individualFitness
            self.totalFitness = self.totalFitness + individualFitness

    def selectParents(self):
        rouletteWheel = []
        wheelSize = self.populationSize * 5
        h_n = []
        for individual in self.population:
            h_n.append(self.population[individual][1])
        j = 0
        for individual in self.population:
            individualLength = round(wheelSize * (h_n[j] / sum(h_n)))
            j += 1
            if individualLength > 0:
                i = 0
                while i < individualLength:
                    rouletteWheel.append(individual)
                    i += 1

        random.shuffle(rouletteWheel)
        parentIndices = []
        i = 0
        while i < self.populationSize:
            parentIndices.append(
                rouletteWheel[random.randint(0, len((rouletteWheel)-1))])
            i += 1

        newGeneration = dict()
        i = 0
        while i < self.populationSize:
            newGeneration[i] = self.population[parentIndices[i]].copy()
            i += 1

        del self.population
        self.population = newGeneration.copy()
        self.updatePopulationFitness()

    def generateChildren(self, crossoverProbability):
        numberOfPairs = round(crossoverProbability * self.populationSize / 2)
        individualIndices = list(range(0, self.populationSize))
        random.shuffle(individualIndices)
        i = 0
        j = 0
        while i < numberOfPairs:
            crossOverPoint = random.randint(0, self.individualSize - 1)
            child1 = self.population[j][0][0:crossOverPoint] + \
                self.population[j+1][0][crossOverPoint:]
            child2 = self.population[j+1][0][0:crossOverPoint] + \
                self.population[j][0][crossOverPoint:]
            self.population[j] = [child1, sum(child1)]
            self.population[j+1] = [child2, sum(child2)]
            i += 1
            j += 2
        self.updatePopulationFitness()

    def mutateChildren(self, mutationProbability):
        numberOfBits = round(mutationProbability *
                             self.populationSize * self.individualSize)
        totalIndices = list(
            range(0, self.populationSize * self.individualSize))
        random.shuffle(totalIndices)
        swapLocations = random.sample(totalIndices, numberOfBits)
        for location in swapLocations:
            individualIndex = math.floor(location / self.individualSize)
            bitIndex = math.floor(location % self.individualSize)
            if self.population[individualIndex][0][bitIndex] == 0:
                self.population[individualIndex][0][bitIndex] = 1
            else:
                self.population[individualIndex][0][bitIndex] = 0

        self.updatePopulationFitness()
        individualSize, populationSize = 8, 10
        i = 0
        instance = GA(individualSize, populationSize)
        while True:
            instance.selectParents()
            instance.generateChildren(0.8)
            instance.mutateChildren(0.03)
            print(instance.population)
            print(instance.totalFitness)
            print(i)
            i += 1
            found = False
            for individual in instance.population:
                if instance.population[individual][1] == individualSize:
                    found = True
                    break
            if found:
                break


In [ ]:
def is_safe(board, row, col):
    for i in range(col):
        if board[row][i] == 1:
            return False

    for i, j in zip(range(row, -1, -1), range(col, -1, -1)):
        if board[i][j] == 1:
            return False

    for i, j in zip(range(row, len(board), 1), range(col, -1, -1)):
        if board[i][j] == 1:
            return False

    return True


def solve_n_queens(board, col):
    if col >= len(board):
        return True

    for i in range(len(board)):
        if is_safe(board, i, col):
            board[i][col] = 1

            if solve_n_queens(board, col + 1):
                return True

            board[i][col] = 0

    return False


def print_solution(board):
    for row in board:
        print(" ".join(map(str, row)))


def solve_8_queens():
    board = [[0 for _ in range(8)] for _ in range(8)]

    if not solve_n_queens(board, 0):
        print("No solution exists.")
    else:
        print("Solution found:")
        print_solution(board)


if __name__ == "__main__":
    solve_8_queens()
